# First predict and select the images to be manually labeled/reviewed

In [ ]:
cd drive/MyDrive/vork/ML/trees/tree-counting-and-classification-in-images/src/utils

In [ ]:
! sh targetted_training.sh /content/drive/MyDrive/vork/ML/trees/train_data_folder/ /content/drive/MyDrive/vork/ML/trees/train_data_to_review/ 20

# Manually label the selected samples via the LblImg tool
In order to do this:
1. Link the selected images from drive, to your computer and start LblImg in that folder.
https://www.google.com/drive/download/
1. Manually label/review them via LblImg.  
This will simultaneously update the labels on drive.
Enter the
```
/content/drive/MyDrive/vork/ML/trees/train_data_to_review/
``` folder and run labelImg to open LabelImg in this folder, then start correcting the predictions.
1. When finished, go to next step: Train on new samples.

# Train on the output from LabelImg
! Split into Train & Valid

In [28]:
train_csv_path = Path('/content/drive/MyDrive/vork/ML/trees/training/training_1/train/labels.csv')
valid_csv_path = Path('/content/drive/MyDrive/vork/ML/trees/training/training_1/valid/labels.csv')

Prepare the csvs from the xmls

In [ ]:
! ls

In [17]:
from glob import glob
from tqdm import tqdm
import pandas as pd
# internal import
import xml_utils


In [34]:
def extract_labels_as_csvs(from_folder_path, to_file):
  xmls_paths = sorted(glob(f"{str(from_folder_path)}/*.xml"))
  accumulator_bboxes_dfs = []
  for xml_path_str in tqdm(xmls_paths, desc="Converting xmls to csv for train eval"):
    xml_path = Path(xml_path_str)
    xml_as_df = xml_utils.xml_to_annotations(str(xml_path))
    accumulator_bboxes_dfs.append(xml_as_df)
  folder_bboxes_df = pd.concat(accumulator_bboxes_dfs)
  folder_bboxes_df.to_csv(to_file, index=False)


In [ ]:
extract_labels_as_csvs(train_csv_path.parent, train_csv_path)
extract_labels_as_csvs(valid_csv_path.parent, valid_csv_path)

In [4]:
# ! pip install -r /content/drive/MyDrive/vork/ML/trees/tree-counting-and-classification-in-images/requirements.txt
# ! pip install --upgrade opencv-python setuptools albumentations==1.0.3

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
#load the modules
import os
import time
import numpy as np
from pathlib import Path
from deepforest import main 
from deepforest import get_data
from deepforest import utilities
from deepforest import preprocess

In [ ]:
m = main.deepforest()
m.use_release()
m.config["epochs"] = 1
m.config["batch_size"] = 2
m.config["save-snapshot"] = False
m.config["train"]["csv_file"] = str(train_csv_path)
m.config["train"]["root_dir"] = str(train_csv_path.parent)
m.config["validation"]["csv_file"] = str(valid_csv_path)
m.config["validation"]["root_dir"] = str(valid_csv_path.parent)

In [ ]:
m.create_trainer()
m.trainer.fit(m)
m.evaluate(csv_file=m.config["validation"]["csv_file"], root_dir=m.config["validation"]["root_dir"])

In [ ]:
# decomment and use when predicting on plaiul fagului
#predicted_raster = model.predict_tile(raster_path, return_plot = True, patch_size=300,patch_overlap=0.25)